In [1]:
!pip install sentence_transformers
# !pip install -U FlagEmbedding

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.7 MB/s eta 0:00:00


In [2]:
import json
import pandas as pd
import numpy as np
# from FlagEmbedding import BGEM3FlagModel
from tqdm.auto import tqdm

In [3]:
path = '/kaggle/input/balanced/data_full_spoiled.json'

In [4]:
df = json.load(open(path))

train = pd.DataFrame(df['train'], columns=['text','intent'])
val = pd.DataFrame(df['val'],columns=['text','intent'])
oos_train = pd.DataFrame(df['oos_train'], columns=['text','intent'])
oos_val = pd.DataFrame(df['oos_val'],columns=['text','intent'])

train['text_len'] = train['text'].map(len)
val['text_len'] = val['text'].map(len)
oos_train['text_len'] = oos_train['text'].map(len)
oos_val['text_len'] = oos_val['text'].map(len)

In [5]:
descriptions = json.load(open('/kaggle/input/new-descriptions2/labels_with_description.json'))

In [6]:
train_df = pd.concat((train, oos_train)).drop(columns='text_len')
val_df = pd.concat((val, oos_val)).drop(columns='text_len')
train_df['description'] = train_df['intent'].map(lambda x: descriptions[x] if x != 'oos' else None)
val_df['description'] = val_df['intent'].map(lambda x: descriptions[x] if x != 'oos' else None)

In [7]:
# model = BGEM3FlagModel('BAAI/bge-m3', use_bfp16=True) 

In [8]:
# n = len(descriptions)
# pairs = []
# nearest_labels = {}

# for i in tqdm(range(n)):
#     embeddings_1 = model.encode(descriptions[i])['dense_vecs']
#     cur_nearest = []
#     max_similarity = 0
#     for j in range(n):
#         if i != j:
#             embeddings_2 = model.encode(descriptions[j])['dense_vecs']
#             similarity = embeddings_1 @ embeddings_2.T
#             if similarity >= max_similarity:
#                 cur_nearest.append(unique_labels[j])
#                 max_similarity = similarity
#                 if len(cur_nearest) > 3:
#                     cur_nearest.pop(0)
                
#     nearest_labels[unique_labels[i]] = cur_nearest

In [11]:
# a = pd.read_csv('/kaggle/input/descriptions-with-context/label_desc_context_new.csv')
# a['nearest_labels'] = a['intent'].map(lambda x: nearest_labels[x] if x != 'oos' else None)
# a.to_csv('label_desc_context_nearest_many.csv', index=False)

In [7]:
uniques = val_df.dropna().groupby('intent').agg('first').reset_index(drop=True)
descriptions = uniques['description'].values

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
def get_inputs(pairs, tokenizer, prompt=None, max_length=1024):
    if prompt is None:
        prompt = "Given a query A and a passage B, determine whether the passage contains an answer to the query by providing a prediction of either 'Yes' or 'No'."
    sep = "\n"
    prompt_inputs = tokenizer(prompt,
                              return_tensors=None,
                              add_special_tokens=False)['input_ids']
    sep_inputs = tokenizer(sep,
                           return_tensors=None,
                           add_special_tokens=False)['input_ids']
    inputs = []
    for query, passage in pairs:
        query_inputs = tokenizer(f'A: {query}',
                                 return_tensors=None,
                                 add_special_tokens=False,
                                 max_length=max_length * 3 // 4,
                                 truncation=True)
        passage_inputs = tokenizer(f'B: {passage}',
                                   return_tensors=None,
                                   add_special_tokens=False,
                                   max_length=max_length,
                                   truncation=True)
        item = tokenizer.prepare_for_model(
            [tokenizer.bos_token_id] + query_inputs['input_ids'],
            sep_inputs + passage_inputs['input_ids'],
            truncation='only_second',
            max_length=max_length,
            padding=False,
            return_attention_mask=False,
            return_token_type_ids=False,
            add_special_tokens=False
        )
        item['input_ids'] = item['input_ids'] + sep_inputs + prompt_inputs
        item['attention_mask'] = [1] * len(item['input_ids'])
        inputs.append(item)
    return tokenizer.pad(
            inputs,
            padding=True,
            max_length=max_length + len(sep_inputs) + len(prompt_inputs),
            pad_to_multiple_of=8,
            return_tensors='pt',
    )

In [12]:
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-v2-gemma')
model = AutoModelForCausalLM.from_pretrained('BAAI/bge-reranker-v2-gemma')
yes_loc = tokenizer('Yes', add_special_tokens=False)['input_ids'][0]
model.eval()

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

In [13]:
model = model.to(device)

In [15]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [16]:
for i in range(25):
    query = uniques.iloc[i]['text']
    pairs = [[query, description] for description in descriptions]
    with torch.no_grad():
        inputs = get_inputs(pairs[:25], tokenizer)
        scores = model(**inputs.to(device), return_dict=True).logits[:, -1, yes_loc].view(-1, ).float()
        print(max(scores))

    gc.collect()
    torch.cuda.empty_cache()

tensor(1.8161, device='cuda:0')
tensor(3.9061, device='cuda:0')
tensor(3.9254, device='cuda:0')
tensor(6.8830, device='cuda:0')
tensor(8.7083, device='cuda:0')
tensor(-1.8232, device='cuda:0')
tensor(-0.6718, device='cuda:0')
tensor(-4.9767, device='cuda:0')
tensor(-1.6897, device='cuda:0')
tensor(3.7225, device='cuda:0')
tensor(2.4931, device='cuda:0')
tensor(2.9321, device='cuda:0')
tensor(-0.5184, device='cuda:0')
tensor(2.1933, device='cuda:0')
tensor(-0.7327, device='cuda:0')
tensor(4.5785, device='cuda:0')
tensor(3.2606, device='cuda:0')
tensor(3.0053, device='cuda:0')
tensor(7.1529, device='cuda:0')
tensor(2.2075, device='cuda:0')
tensor(0.2899, device='cuda:0')
tensor(-1.2381, device='cuda:0')
tensor(2.9346, device='cuda:0')
tensor(5.4343, device='cuda:0')


KeyboardInterrupt: 